# Lab 3 - Model Deplyoment 

In this lab, you will learn how to use Azure Machine Learning Service to deploy, manage, and monitor the trained models.


The following diagram illustrates the complete deployment workflow.

![AML Arch](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/model-ci-cd.png)



The deployment workflow includes the following steps:

- Create/Retrain the model
- Register the model in a registry hosted in your Azure Machine Learning Service workspace
- Register an image that pairs a model with a scoring script and dependencies in a portable container
- Deploy the image as a web service in the cloud or to edge devices
- Monitor and collect data


You completed the first two steps in the previous labs.

In this lab we will walk-through the reminder of the deployment workflow.



## Connect to the workspace

In [ ]:
# Verify AML SDK Installed
# view version history at https://pypi.org/project/azureml-sdk/#history 
import azureml.core
print("SDK Version:", azureml.core.VERSION)

In [ ]:
from azureml.core import Workspace

# Read the workspace config from file
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

## Create and deploy the container image encapsulating the model

When you deploy a model using AML to either ACI or AKS, you are deploying a Docker container encapsulating a trained model, its dependencies, and a web services wrapper around the model. 

### Create scoring script
Create the scoring script, called score.py, used by the web service call to invoke the model.

You must include two required functions in the scoring script:

- The `init()` function, which loads the model into a global object. This function is run only once when the Docker container is started.

- The `run(input_data)` function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats can be used.

In [ ]:
%%writefile score.py

import json
import os
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.externals import joblib

from azureml.core.model import Model
from azureml.core import Workspace

def init():
    try:
        global model   

        model_name = '<<modelid>>'
        model_path = Model.get_model_path(model_name)
        model = joblib.load(model_path)
        
    except Exception as e:
        print('Exception during init: ', str(e))

def run(input_json):     
    try:
        inputs = json.loads(input_json)
        prediction = model.predict(inputs)
        prediction = json.dumps(prediction.tolist())

    except Exception as e:
        prediction = str(e)
    return prediction


Substitute the actual model ID in the script file.

In [ ]:
from azureml.core.model import Model

model_name = 'propensity_to_buy_predictor'
model = Model(ws, name=model_name)
script_file_name = 'score.py'

with open(script_file_name, 'r') as cefr:
    content = cefr.read()
    
with open(script_file_name, 'w') as cefw:
    cefw.write(content.replace('<<modelid>>', model.name))

Review the updated script.

In [ ]:
with open("score.py","r") as f:
    print(f.read())

### Create a Conda dependencies environment file.

Next, create an environment file that specifies the script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image. 


You need to replace the values in `experiment_name`  with the name of your experiment.


In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

mycondaenv = CondaDependencies.create(conda_packages=['scikit-learn','numpy','pandas'])

with open("mydeployenv.yml","w") as f:
    f.write(mycondaenv.serialize_to_string())

Review the content of 'yml' file.

In [ ]:
with open("mydeployenv.yml","r") as f:
    print(f.read())

### Create docker image for deployment

To create a Container Image, you need four things: the model metadata (as retrieved from Model Registry), the scoring script file, the runtime configuration (defining whether Python or PySpark should be used) and the Conda Dependencies file.

In [ ]:
from azureml.core.image import ContainerImage, Image

# Define runtime
runtime = "python" 

# Define scoring script
driver_file = "score.py"

# Define conda dependencies
conda_file = "mydeployenv.yml"

# configure the image
image_config = ContainerImage.image_configuration(execution_script=driver_file, 
                                                  runtime=runtime, 
                                                  conda_file=conda_file,
                                                  description="Image for propensity to buy predictor",
                                                  tags={"Classifier": "AutomatedML"})

image = Image.create(name = "propensity-to-buy-classifier",
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

image.wait_for_creation(show_output = True)


### (Optional) Download the Docker image from AML service

You can download the docker image that gets deployed to the Azure container registry to your local docker environment. You need to have Docker installed in your local environment to be able to run the following commands. You also need to install the azure cli. https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest

To download (or pull) the docker image from the Azure container registry to your local docker registry run the following code from a terminal.

In [ ]:
az acr login --name <registry_name>
docker pull (image_name:tag>
            - pulls it locally to your docker registry
docker images
             - lists the docker image you just pulled locally
docker run -i -t <image_name:tag> /bin/sh 
             - logs into the shell of the image

### Deploy the container image to ACI

With the Container Image  in hand, you are almost ready to deploy to ACI. The next step is to define the size of the VM that ACI will use to run your Container.

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 1, 
    tags = {'name':'Azure ML ACI'}, 
    description = 'This is a deployment of the propensity to buy predictor.')

At this point you can deploy the image to the webservice to ACI

In [ ]:
from azureml.core.webservice import Webservice

service_name = "propensity-to-buy-predictor-aci"
print("Deploying: ", service_name)
aci_service = Webservice.deploy_from_image(deployment_config = aci_config,
                                           image = image,
                                           name = service_name,
                                           workspace = ws)
aci_service.wait_for_deployment(True)

#print(aci_service.get_logs())

### Test the service

Once the webservice deployment completes, you can use the returned webservice object to invoke the webservice. 

#### Load test data

In [ ]:
import numpy as np
import pandas as pd
import os

# Load a test dataset
folder = '../datasets'
filename = 'banking_test.csv'
pathname = os.path.join(folder, filename)
df_test = pd.read_csv(pathname, delimiter=',')
feature_columns = [
                   # Demographic
                   'age', 
                   'job', 
                   'education', 
                   'marital',  
                   'housing', 
                   'loan', 
                   # Previous campaigns
                   'month',
                   'campaign',
                   'poutcome',
                   # Economic indicators
                   'emp_var_rate',
                   'cons_price_idx',
                   'cons_conf_idx',
                   'euribor3m',
                   'nr_employed']
df_test = df_test[feature_columns]
df_test = pd.get_dummies(df_test, drop_first=True).astype(dtype='float')


#### Invoke the service

In [ ]:
import json

test_data = json.dumps(df_test[0:10].values.tolist())
print(test_data)
result = aci_service.run(input_data = test_data)
print("Result:")
print(result)

#### Clean up

In [ ]:
aci_service.delete()

### Deploy the container image to AKS

Once you are familiar with the process for deploying a webservice to ACI, you will find the process for deploying to AKS to be similar with one additional step that creates the AKS cluster first.

In [ ]:
# Provision an AKS cluster 

from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice

# Use the default configuration, overriding the default location to a known region that supports AKS
prov_config = AksCompute.provisioning_configuration(location='westus2')

aks_name = 'aks-cluster01' 

# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)


# Wait for cluster to be ready
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

With your AKS cluster ready, now you can deploy your webservice. Once again, you need to provide a configuration for the size of resources allocated from the AKS cluster to run instances of your Container.

In [ ]:
from azureml.core.image import ContainerImage, Image

images = Image.list(ws, image_name="propensity-to-buy-classifier")
images
image = images[0]

In [ ]:
# Create the web service configuration (using defaults)
aks_config = AksWebservice.deploy_configuration()

aks_service_name ='propensity-to-buy-predictor-aks'

aks_service = Webservice.deploy_from_image(
  workspace=ws, 
  name=aks_service_name, 
  image = image,
  deployment_target=aks_target
  )


aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

### Test the service
As before, you can use the webservice object returned by the deploy_from_model method to invoke your deployed webservice. 

In [ ]:
import json

test_data = json.dumps(df_test[0:10].values.tolist())
result = aks_service.run(input_data = test_data)

print(result)

## Clean up

Make sure to remove ACI and AKS deployments. Use Azure Portal to remove *Deployments* and *AKS Compute*.